In [1]:
import tensorflow as tf
import scipy.io as sio
import numpy as np

In [2]:
hello = tf.constant('Hello, TensorFlow!')
result = tf.one_hot([9, 2, 3, 4], 10)
sess = tf.Session()
print(sess.run(result))

[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.]]


In [61]:
# Importa todas as imagens e etiquetas
mat = sio.loadmat("train_32x32.mat")
labels, images = mat['y'], mat['X']

# Reformata as imagens para 2 dimensoes (pixel, imagem)
images = np.swapaxes(images, 0, 3)
images = np.swapaxes(images, 2, 3)

# Reformata labels para one hot enconding
labels = (np.arange(10) == labels[:]).astype(np.float32)-1

# Reformata labels para 1-D
#labels = labels.reshape((len(labels)))

In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [46]:
num_images = len(images[:,0,0,0])
image_size = 32
filter_size = 3
num_categories = num_labels = 10
batch_size = 100
num_channels = 3
layers_size = image_size*image_size*num_channels
num_steps = 1000

In [53]:
graph2 = tf.Graph()
with graph2:
    
    input_layer = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    input_label = tf.placeholder(tf.int32, shape=(batch_size))
    
    filter1 = tf.Variable(tf.truncated_normal([batch_size, filter_size, filter_size, num_channels]), dtype=tf.float32)
    net11 = tf.nn.conv2d(input_layer, filter1, strides=[1, 1, 1, 1], padding='SAME')
    net12 = tf.nn.max_pool(net11, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    layer1 = tf.nn.relu(net12)
    
    filter2 = tf.Variable(tf.truncated_normal([batch_size, filter_size, filter_size, num_channels], dtype=tf.float32))
    net21 = tf.nn.conv2d(layer1, filter2, strides=[1, 1, 1, 1], padding='SAME')
    net22 = tf.nn.max_pool(net21, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    layer2 = tf.nn.relu(net22)
    
    shape = layer2.get_shape().as_list()
    layer2_reshaped = tf.reshape(layer2, [shape[0], shape[1] * shape[2] * shape[3]])
    
    weights3 = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * num_channels, 64], dtype=tf.float32))
    layer3 = tf.nn.relu(tf.matmul(layer2_reshaped, weights3))
    
    weights4 = tf.Variable(tf.truncated_normal([64, num_categories], dtype=tf.float32))
    output = tf.matmul(layer3, weights4)
    
    erro = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=output, labels=input_label)
    loss = tf.reduce_mean(erro)
    
    prediction = tf.nn.softmax(output)
    
    optimizer = tf.train.GradientDescentOptimizer(0.01).minimize(loss)

AttributeError: __exit__

In [59]:
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()
with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

In [65]:
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    for step in range(num_steps):
        offset = np.random.randint(0, num_images-batch_size)
        train_im = images[offset:offset+batch_size,:,:,:]
        train_l = labels[offset:offset+batch_size,:]
                        
        _, l = sess.run([optimizer, loss], feed_dict={tf_train_dataset:train_im, tf_train_labels:train_l})
        
        print step, l

0 -1556.78
1 -1.57018e+12
2 nan
3 nan
4 nan
5 nan
6 nan
7 nan
8 nan
9 nan
10 nan
11 nan
12 nan
13 nan
14 nan
15 nan
16 nan
17 nan
18 nan
19 nan
20 nan
21 nan
22 nan
23 nan
24 nan
25 nan
26 nan
27 nan
28 nan
29 nan
30 nan
31 nan
32 nan
33 nan
34 nan
35 nan
36 nan
37 nan
38 nan
39 nan
40 nan
41 nan
42 nan
43 nan
44 nan
45 nan
46 nan
47 nan
48 nan
49 nan
50 nan
51 nan
52 nan
53 nan
54 nan
55 nan
56 nan
57 nan


KeyboardInterrupt: 